In [2]:
from utils.binaries import *
from utils.plotting import *
import pickle

import polars as pl

from utils.Auger.SD.Triggers import filter_and_downsample, wcd_t1_trigger

14:56:31 (   +9.0s) [INFO   ] -- import logging
14:56:31 (  +160ms) [INFO   ] -- import numpy as np
14:56:31 (    +7ms) [INFO   ] -- import uncertainties
14:56:31 (  +208ms) [INFO   ] -- import pandas as pd
14:56:31 (    +2ms) [WARNING] -- pandas support to be dropped at some point!
14:56:31 (    +1ms) [INFO   ] -- import binaries.tools as tools
14:56:31 (  +285ms) [INFO   ] -- import matplotlib.pyplot as plt
14:56:32 (  +430ms) [INFO   ] -- import seaborn as so
14:56:32 (    +6ms) [DEBUG  ] -- font size set to 9.5
14:56:32 (    +2ms) [DEBUG  ] -- label size set to 13.0
14:56:32 (    +0ms) [DEBUG  ] -- figure size set to [6.6, 3.3]
14:56:32 (    +1ms) [DEBUG  ] -- markersize set to 2.0
14:56:32 (    +1ms) [DEBUG  ] -- usetex set to False
14:56:32 (    +2ms) [INFO   ] -- import plotting.tools as plot


In [3]:
def read_file(path_to_file):

    trace_has_ssd = lambda df: len(df.filter(pl.col("id") == 5)) == 1
    trace_has_all_wcd = (
        lambda df: len(
            df.filter(
                (pl.col("pmt") == 1) | (pl.col("pmt") == 2) | (pl.col("pmt") == 3)
            )
        )
        == 3
    )

    data = pl.read_csv(
        path_to_file,
        has_header=False,
        skip_rows=1,
        new_columns=["id", "pmt", "baseline"],
    )

    data = data.with_columns(
        zero_trace=pl.concat_list(
            pl.exclude("id", "pmt", "baseline") - pl.col("baseline")
        )
    )
    data = data.select("id", "pmt", "baseline", "zero_trace")

    for _id, pmt, baseline, trace in data.iter_rows():
        if sum([_bin is None for _bin in trace]):
            data = data.filter(pl.col("id") != _id)

    return data.group_by(["id"])


def clip_zero_signal(trace):

    pmt1, pmt2, pmt3 = trace
    assert (
        (trace_length := len(pmt1)) == len(pmt2) == len(pmt3)
    ), "Unrealistic time trace"

    for start in range(trace_length):
        if pmt1[start] != pmt2[start] != pmt3[start] != 0:
            break
    for stop in range(-1, -trace_length, -1):
        if pmt1[stop] != pmt2[stop] != pmt3[stop] != 0:
            break

    return np.array(trace)[:, max(start - 10, 0) : min(stop + 10, trace_length)]


def plot_test_trace(trace, ax):
    pmt1, pmt2, pmt3 = trace
    ax.plot(range(len(pmt1)), pmt1, label="pmt1")
    ax.plot(range(len(pmt1)), pmt2, label="pmt2")
    ax.plot(range(len(pmt1)), pmt3, label="pmt3")

In [4]:
# from /cr/data01/filip/offline/Framework/SDetector/SdSimCalibrationConstants.xml.in
vem_peak = np.array([215.781, 215.781, 215.781])
mip_peak = 51.8

trace_has_all_wcd = lambda df: len(get_wcd_traces(df)) == 3

get_wcd_traces = lambda df: df.filter(
    (pl.col("pmt") == 1) | (pl.col("pmt") == 2) | (pl.col("pmt") == 3)
).to_numpy()[:, 3:][:, 0]

read_file = lambda path: pl.read_csv(
    path,
    has_header=False,
    skip_rows=1,
    new_columns=["id", "pmt", "baseline"],
)

import os

run = "run4"
base = f"/cr/tempdata01/filip/LESimulations/{run}"
for step, file in enumerate(os.listdir(base), 1):

    if not file.startswith("x"):
        continue

    print(f"{step}/{len(os.listdir(base))}", end="\r")
    # if step >= 10: break

    data = read_file(f"{base}/{file}")
    data = data.with_columns(
        zero_trace=pl.concat_list(
            pl.exclude("id", "pmt", "baseline") - pl.col("baseline")
        )
    )
    data = data.select("id", "pmt", "baseline", "zero_trace")

    for _id, pmt, baseline, trace in data.iter_rows():
        if sum([_bin is None for _bin in trace]):
            data = data.filter(pl.col("id") != _id)

    data.rechunk()

    events = data["id"].unique()
    traces = data.group_by(["id"])

    t1_peaks, all_peaks = [], []
    for _id, trace in traces:
        if len(trace.filter(pl.col("pmt") == 5)) == 0:
            continue  # no signal in SSD

        if not trace_has_all_wcd(trace):
            t1_latch_bin = -1  # not all WCD PMTs received signal -> no T1
        else:
            wcd_traces = get_wcd_traces(trace)
            filtered_downsampled = filter_and_downsample(*wcd_traces)
            # fds_clipped_trace = clip_zero_signal(filtered_downsampled)
            vem_trace = np.array(filtered_downsampled) / vem_peak[:, np.newaxis]

            t1_latch_bin = wcd_t1_trigger(vem_trace, latch_bin=True)

        ssd_trace = (
            trace.filter(pl.col("pmt") == 5)["zero_trace"].to_numpy()[0] / mip_peak
        )

        if t1_latch_bin != -1:
            t1_latch_bin *= 3  # to convert from UUB to UB binning
            ssd_max = ssd_trace[
                max(0, t1_latch_bin - 19) : min(t1_latch_bin + 40, len(ssd_trace))
            ].max()
            t1_peaks.append(ssd_max)
        else:
            ssd_max = ssd_trace.max()

        all_peaks.append(ssd_max)

    with open(
        f"/cr/tempdata01/filip/LESimulations/processed/{run}_{file}_all.pkl", "wb"
    ) as f:
        pickle.dump(all_peaks, f)
    with open(
        f"/cr/tempdata01/filip/LESimulations/processed/{run}_{file}_t1.pkl", "wb"
    ) as f:
        pickle.dump(t1_peaks, f)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)

plot_test_trace(np.array(wcd_traces) / vem_peak[:, np.newaxis], ax1)
# plot_test_trace(vem_trace, ax2)
# ax2.plot(range(len(ssd_trace) // 3), ssd_trace[::3])
ax1.plot(range(len(ssd_trace)), np.array(ssd_trace))